In [1]:
# pip install jellyfish

In [2]:
# pip install levenshtein

In [3]:
# pip install textdistance

In [4]:
# pip install fuzzywuzzy

In [18]:
# pip install epitran

In [3]:
pip install unicodecsv

Defaulting to user installation because normal site-packages is not writeable
  Using cached unicodecsv-0.14.1.tar.gz (10 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [1 lines of output]
  ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [5]:
import pandas as pd
import numpy as np
import jellyfish
from Levenshtein import distance
import textdistance
from fuzzywuzzy import fuzz

In [12]:
input_bn = 'PLDT'

In [13]:
df_bn = pd.read_csv('./data/company.csv', on_bad_lines='skip')[['Company Name ']]
df_bn['levenshtein'] = df_bn['Company Name '].apply(lambda x: jellyfish.levenshtein_distance(input_bn, x))
df_bn['soundex'] = df_bn['Company Name '].apply(lambda x: fuzz.ratio(jellyfish.soundex(input_bn), 
                                                                     jellyfish.soundex(x)))
df_bn['metaphone'] = df_bn['Company Name '].apply(lambda x: fuzz.ratio(jellyfish.metaphone(input_bn), 
                                                                       jellyfish.metaphone(x)))
df_bn

,Company Name,levenshtein,soundex,metaphone
0,"8990 Holdings, Inc. Series B Perpetual Preferr...",54,50,17
1,"8990 Holdings, Inc. Series A Perpetual Preferr...",54,50,17
2,Asia Amalgamated Holdings Corporation,37,25,21
3,"Atok-Big Wedge Co., Inc.",24,25,11
4,"AbaCore Capital Holdings, Inc.",30,0,25
...,...,...,...,...
282,"Wilcon Depot, Inc.",17,25,38
283,"Waterfront Philippines, Incorporated",35,25,23
284,Xurpas Inc.,11,0,17
285,NexGen Energy Corp.,19,0,11


In [14]:
df_bn.sort_values('levenshtein').head(20)

,Company Name,levenshtein,soundex,metaphone
265,PLDT Inc.,5,75,67
214,PT&T Corp.,8,50,55
151,LMG Corp.,9,0,18
91,PLDT INC. -USD,10,75,53
83,"DFNN, Inc.",10,0,18
194,"Paxys, Inc.",10,25,17
178,"MREIT, Inc.",10,25,18
33,"AREIT, Inc.",10,25,18
284,Xurpas Inc.,11,0,17
137,"Ionics, Inc.",12,0,0


In [15]:
df_bn.sort_values('soundex', ascending=False).head(20)

,Company Name,levenshtein,soundex,metaphone
193,"PAL Holdings, Inc.",16,75,35
269,PLDT Inc. Convertible Preferred Shares Series ...,48,75,22
268,PLDT Inc. Convertible Preferred Shares Series ...,48,75,23
91,PLDT INC. -USD,10,75,53
265,PLDT Inc.,5,75,67
128,"8990 Holdings, Inc.",19,50,29
191,The Philodrill Corporation,25,50,22
173,Mabuhay Holdings Corporation,28,50,19
146,"Kepwealth Property Phils., Inc.",30,50,26
1,"8990 Holdings, Inc. Series A Perpetual Preferr...",54,50,17


In [16]:
df_bn.sort_values('metaphone', ascending=False).head(20)

,Company Name,levenshtein,soundex,metaphone
265,PLDT Inc.,5,75,67
214,PT&T Corp.,8,50,55
91,PLDT INC. -USD,10,75,53
202,DigiPlus Interactive Corp.,25,25,38
282,"Wilcon Depot, Inc.",17,25,38
22,"AllDay Marts, Inc.",17,50,38
193,"PAL Holdings, Inc.",16,75,35
105,Filinvest REIT Corp.,19,25,35
138,"IPM Holdings, Inc.",17,25,33
139,"iPeople, Inc.",12,25,33


In [11]:
# different_rows = df_bn[df_bn['levenshtein'] != df_bn['phonetics']]
# different_rows